# Transfer Learning: 3-Way Comparison with Memorized Model Control

**Complete analysis testing whether grokked mechanisms specifically enable transfer**

## Three Conditions:

1. **Transfer from GROKKED model** ⭐
   - Addition model: LOW train loss + LOW test loss (generalized)
   - Tests: Do generalizing mechanisms transfer?

2. **Transfer from MEMORIZED model** 🔬 **NEW CONTROL**
   - Addition model: LOW train loss + HIGH test loss (memorized, not generalized)
   - Tests: Does mere memorization help? Or only generalization?

3. **Random initialization** (baseline)
   - No prior knowledge
   - Tests: Starting from scratch

## Key Research Question:
**Is it specifically the grokked (generalized) mechanisms that transfer, or does any training help?**

If grokking matters: Grokked >> Memorized ≈ Random  
If any training helps: Grokked ≈ Memorized >> Random  

**Expected Runtime:** ~8-10 hours on GPU (5 seeds × 3 conditions)

## Configuration

In [ ]:
# ========== EXPERIMENT CONFIGURATION ==========

NUM_SEEDS = 5
SEEDS = [42, 123, 456, 789, 1024]

# Training epochs per condition
TRANSFER_EPOCHS = 10000   # Transfer conditions
BASELINE_EPOCHS = 30000   # Random baseline needs time to grok

# Accuracy thresholds
THRESHOLDS = [0.90, 0.95, 0.99, 0.999]

# Checkpoints
GROKKED_CHECKPOINT = 'saved_runs/wd_10-1_mod_addition_loss_curve.pth'
MEMORIZED_CHECKPOINT = None  # Will be determined or trained

# For training memorized model if needed
MEMORIZED_TRAIN_EPOCHS = 2000  # Enough to memorize, not enough to grok

print("="*80)
print("3-WAY TRANSFER LEARNING COMPARISON")
print("="*80)
print(f"\nSeeds: {SEEDS} (N={NUM_SEEDS})")
print(f"\nConditions:")
print(f"  1. Grokked transfer    : {TRANSFER_EPOCHS:,} epochs")
print(f"  2. Memorized transfer  : {TRANSFER_EPOCHS:,} epochs (NEW CONTROL)")
print(f"  3. Random baseline     : {BASELINE_EPOCHS:,} epochs")
print(f"\nThresholds: {[f'{t:.1%}' for t in THRESHOLDS]}")
print(f"\nEstimated runtime: ~8-10 hours on GPU")
print(f"Total experiments: {NUM_SEEDS * 3} ({NUM_SEEDS} seeds × 3 conditions)")

## Setup

In [ ]:
from google.colab import drive
import os
from datetime import datetime
import time

drive.mount('/content/drive')

timestamp = datetime.now().strftime('%Y%m%d_%H%M%S')
DRIVE_BASE = '/content/drive/MyDrive/grokking_transfer_experiments'
EXPERIMENT_DIR = f'{DRIVE_BASE}/3way_run_{timestamp}'

os.makedirs(EXPERIMENT_DIR, exist_ok=True)
os.makedirs(f'{EXPERIMENT_DIR}/figures', exist_ok=True)
os.makedirs(f'{EXPERIMENT_DIR}/checkpoints', exist_ok=True)
os.makedirs(f'{EXPERIMENT_DIR}/results', exist_ok=True)

print(f"✓ Experiment directory: {EXPERIMENT_DIR}")

In [ ]:
# Clone and setup
if not os.path.exists('progress-measures-paper-extension'):
    !git clone https://github.com/Junekhunter/progress-measures-paper-extension.git
os.chdir('progress-measures-paper-extension')

!pip install -q einops

import sys
sys.path.insert(0, '.')

import torch
import torch.nn as nn
import torch.optim as optim
import numpy as np
import matplotlib.pyplot as plt
from dataclasses import replace
import random
from pathlib import Path
from tqdm import tqdm
import json

from transformers import Transformer, Config, gen_train_test, full_loss
import helpers

print("\n✓ Imports successful!")
print(f"PyTorch: {torch.__version__}")
print(f"CUDA: {torch.cuda.is_available()}")
if torch.cuda.is_available():
    print(f"Device: {torch.cuda.get_device_name(0)}")

## Load or Create Memorized Model

In [ ]:
# Load grokked checkpoint
print(f"Loading GROKKED checkpoint: {GROKKED_CHECKPOINT}")
grokked_checkpoint = torch.load(GROKKED_CHECKPOINT, map_location='cpu')

addition_config = Config(
    lr=1e-3,
    weight_decay=1.0,
    p=113,
    d_model=128,
    fn_name='add',
    frac_train=0.3,
    num_epochs=50000,
    seed=0,
    device=torch.device('cuda' if torch.cuda.is_available() else 'cpu')
)

# Load grokked model
grokked_model = Transformer(addition_config, use_cache=False)
grokked_model.to(addition_config.device)

if 'model' in grokked_checkpoint:
    grokked_model.load_state_dict(grokked_checkpoint['model'])
    print("✓ Loaded grokked model from 'model' key")
elif 'state_dicts' in grokked_checkpoint:
    grokked_model.load_state_dict(grokked_checkpoint['state_dicts'][-1])
    print(f"✓ Loaded grokked model from 'state_dicts'[-1]")

# Verify it's grokked
if 'test_losses' in grokked_checkpoint:
    final_test = grokked_checkpoint['test_losses'][-1]
    print(f"  Grokked model test loss: {final_test:.6f} {'✓' if final_test < 0.01 else '⚠'}")

In [ ]:
# Find or create memorized (non-grokked) checkpoint
print(f"\nFinding/creating MEMORIZED checkpoint...")
print("  Target: LOW train loss (<0.1) + HIGH test loss (>1.0)")
print("="*80)

memorized_model = None
memorized_epoch = None
memorized_train_loss = None
memorized_test_loss = None

# Check if checkpoint has intermediate states
if 'state_dicts' in grokked_checkpoint and 'train_losses' in grokked_checkpoint:
    print("Searching through checkpoint history...")
    train_losses = grokked_checkpoint['train_losses']
    test_losses = grokked_checkpoint['test_losses']
    state_dicts = grokked_checkpoint['state_dicts']
    
    # Find best memorized checkpoint
    best_idx = None
    for i in range(len(train_losses)):
        if train_losses[i] < 0.1 and test_losses[i] > 1.0:
            if i < len(state_dicts):  # Make sure we have the state dict
                best_idx = i
                break  # Take first one that meets criteria
    
    if best_idx is not None:
        memorized_model = Transformer(addition_config, use_cache=False)
        memorized_model.load_state_dict(state_dicts[best_idx])
        memorized_model.to(addition_config.device)
        memorized_epoch = best_idx
        memorized_train_loss = train_losses[best_idx]
        memorized_test_loss = test_losses[best_idx]
        
        print(f"✓ Found memorized checkpoint at epoch {best_idx}")
        print(f"  Train loss: {memorized_train_loss:.6f} (memorized training set)")
        print(f"  Test loss:  {memorized_test_loss:.6f} (NOT generalized)")
    else:
        print("  No suitable intermediate checkpoint found")

# If not found, train a new memorized model
if memorized_model is None:
    print(f"\nTraining new memorized model ({MEMORIZED_TRAIN_EPOCHS} epochs)...")
    print("  Goal: Memorize training set without generalizing")
    
    memorized_model = Transformer(addition_config, use_cache=False)
    memorized_model.to(addition_config.device)
    
    optimizer = optim.AdamW(memorized_model.parameters(), 
                           lr=addition_config.lr, 
                           weight_decay=addition_config.weight_decay,
                           betas=(0.9, 0.98))
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda step: min(step/10, 1))
    
    train_data, test_data = gen_train_test(addition_config)
    
    train_losses_mem = []
    test_losses_mem = []
    
    pbar = tqdm(range(MEMORIZED_TRAIN_EPOCHS), desc="Training memorized model")
    for epoch in pbar:
        train_loss = full_loss(addition_config, memorized_model, train_data)
        train_loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        with torch.no_grad():
            test_loss = full_loss(addition_config, memorized_model, test_data)
        
        train_losses_mem.append(train_loss.item())
        test_losses_mem.append(test_loss.item())
        
        if epoch % 100 == 0:
            pbar.set_postfix({'train': f'{train_loss.item():.4f}', 
                            'test': f'{test_loss.item():.4f}'})
    
    memorized_epoch = MEMORIZED_TRAIN_EPOCHS
    memorized_train_loss = train_losses_mem[-1]
    memorized_test_loss = test_losses_mem[-1]
    
    print(f"\n✓ Trained memorized model for {MEMORIZED_TRAIN_EPOCHS} epochs")
    print(f"  Train loss: {memorized_train_loss:.6f}")
    print(f"  Test loss:  {memorized_test_loss:.6f}")
    
    # Save this memorized checkpoint
    torch.save({
        'model': memorized_model.state_dict(),
        'train_loss': memorized_train_loss,
        'test_loss': memorized_test_loss,
        'epoch': memorized_epoch,
        'config': addition_config
    }, f'{EXPERIMENT_DIR}/memorized_addition_model.pth')
    print(f"  Saved to {EXPERIMENT_DIR}/memorized_addition_model.pth")

print("\n" + "="*80)
print("MODEL COMPARISON:")
print("="*80)
print(f"Grokked model:    train_loss ≈ 0.000, test_loss ≈ 0.000 (generalized)")
print(f"Memorized model:  train_loss = {memorized_train_loss:.6f}, test_loss = {memorized_test_loss:.6f} (NOT generalized)")
print(f"\n✓ Both source models ready for transfer experiments!")

## Training Function

In [ ]:
def train_with_tracking(model, config, num_epochs, condition_name, seed_label):
    """Train model with comprehensive tracking."""
    model.to(config.device)
    model.train()
    
    optimizer = optim.AdamW(model.parameters(), lr=config.lr, 
                           weight_decay=config.weight_decay, betas=(0.9, 0.98))
    scheduler = optim.lr_scheduler.LambdaLR(optimizer, lambda step: min(step/10, 1))
    
    train_data, test_data = gen_train_test(config)
    
    train_losses = []
    test_losses = []
    test_accuracies = []
    threshold_epochs = {t: None for t in THRESHOLDS}
    grokking_epoch = None
    
    start_time = time.time()
    pbar = tqdm(range(num_epochs), desc=f"{condition_name} {seed_label}")
    
    for epoch in pbar:
        train_loss = full_loss(config, model, train_data)
        train_loss.backward()
        optimizer.step()
        scheduler.step()
        optimizer.zero_grad()
        
        with torch.no_grad():
            test_loss = full_loss(config, model, test_data)
            test_tensor = torch.tensor(test_data).to(config.device)
            logits = model(test_tensor)[:, -1]
            predictions = logits.argmax(dim=-1)
            labels = torch.tensor([config.fn(i, j) for i, j, _ in test_data]).to(config.device)
            test_accuracy = (predictions == labels).float().mean().item()
        
        train_losses.append(train_loss.item())
        test_losses.append(test_loss.item())
        test_accuracies.append(test_accuracy)
        
        for threshold in THRESHOLDS:
            if threshold_epochs[threshold] is None and test_accuracy >= threshold:
                threshold_epochs[threshold] = epoch
                if threshold == 0.999:
                    print(f"\n🎯 {condition_name} reached 99.9% at epoch {epoch}!")
        
        # Grokking detection
        if grokking_epoch is None and epoch >= 100:
            recent_avg = np.mean(test_losses[epoch-100:epoch-10])
            current_avg = np.mean(test_losses[epoch-10:epoch])
            if recent_avg - current_avg > 1.0:
                grokking_epoch = epoch
                print(f"\n⚡ {condition_name} grokking at epoch {epoch}!")
        
        if epoch % 100 == 0:
            pbar.set_postfix({'acc': f'{test_accuracy:.4f}', 
                            'train': f'{train_loss.item():.4f}'})
    
    return {
        'train_losses': train_losses,
        'test_losses': test_losses,
        'test_accuracies': test_accuracies,
        'threshold_epochs': threshold_epochs,
        'grokking_epoch': grokking_epoch,
        'final_test_accuracy': test_accuracies[-1],
        'training_time': time.time() - start_time,
        'model_state': model.state_dict(),
        'seed': config.seed,
        'condition': condition_name
    }

print("✓ Training function defined!")

## Run 3-Way Experiments

In [ ]:
all_grokked_results = []
all_memorized_results = []
all_random_results = []

experiment_start = time.time()

print("\n" + "="*80)
print(f"STARTING 3-WAY EXPERIMENTS")
print(f"Total: {NUM_SEEDS * 3} experiments")
print("="*80)

for i, seed in enumerate(SEEDS):
    print(f"\n{'='*80}")
    print(f"SEED {i+1}/{NUM_SEEDS}: {seed}")
    print(f"{'='*80}")
    
    subtraction_config = replace(addition_config, fn_name='subtract', seed=seed)
    
    # ===== 1. GROKKED TRANSFER =====
    print(f"\n✨ GROKKED TRANSFER (seed {seed})")
    print("-" * 80)
    
    grokked_transfer_model = Transformer(subtraction_config, use_cache=False)
    grokked_transfer_model.load_state_dict(grokked_model.state_dict())
    
    grokked_results = train_with_tracking(
        grokked_transfer_model, subtraction_config, 
        TRANSFER_EPOCHS, 'grokked_transfer', f"seed {seed}"
    )
    all_grokked_results.append(grokked_results)
    
    print(f"✓ Grokked transfer: {grokked_results['final_test_accuracy']:.4f} final accuracy")
    torch.save(grokked_results, f"{EXPERIMENT_DIR}/checkpoints/grokked_transfer_seed{seed}.pth")
    
    del grokked_transfer_model
    torch.cuda.empty_cache()
    
    # ===== 2. MEMORIZED TRANSFER =====
    print(f"\n🧠 MEMORIZED TRANSFER (seed {seed}) [NEW CONTROL]")
    print("-" * 80)
    
    memorized_transfer_model = Transformer(subtraction_config, use_cache=False)
    memorized_transfer_model.load_state_dict(memorized_model.state_dict())
    
    memorized_results = train_with_tracking(
        memorized_transfer_model, subtraction_config,
        TRANSFER_EPOCHS, 'memorized_transfer', f"seed {seed}"
    )
    all_memorized_results.append(memorized_results)
    
    print(f"✓ Memorized transfer: {memorized_results['final_test_accuracy']:.4f} final accuracy")
    torch.save(memorized_results, f"{EXPERIMENT_DIR}/checkpoints/memorized_transfer_seed{seed}.pth")
    
    del memorized_transfer_model
    torch.cuda.empty_cache()
    
    # ===== 3. RANDOM BASELINE =====
    print(f"\n🎲 RANDOM BASELINE (seed {seed})")
    print("-" * 80)
    
    random_model = Transformer(subtraction_config, use_cache=False)
    
    random_results = train_with_tracking(
        random_model, subtraction_config,
        BASELINE_EPOCHS, 'random_baseline', f"seed {seed}"
    )
    all_random_results.append(random_results)
    
    print(f"✓ Random baseline: {random_results['final_test_accuracy']:.4f} final accuracy")
    torch.save(random_results, f"{EXPERIMENT_DIR}/checkpoints/random_baseline_seed{seed}.pth")
    
    del random_model
    torch.cuda.empty_cache()
    
    # Progress
    done = (i + 1) * 3
    total = NUM_SEEDS * 3
    elapsed = time.time() - experiment_start
    remaining = (elapsed / done) * (total - done)
    
    print(f"\n📊 Progress: {done}/{total} experiments")
    print(f"   Elapsed: {elapsed/3600:.1f}h | Remaining: ~{remaining/3600:.1f}h")

total_time = time.time() - experiment_start
print(f"\n{'='*80}")
print("✅ ALL 3-WAY EXPERIMENTS COMPLETE!")
print(f"Total time: {total_time/3600:.2f} hours")
print(f"{'='*80}")

## 3-Way Statistical Analysis

In [ ]:
from scipy import stats

def calc_stats(values):
    values = [v for v in values if v is not None]
    if len(values) == 0:
        return None, None, None, None, 0
    mean = np.mean(values)
    std = np.std(values, ddof=1) if len(values) > 1 else 0
    if len(values) > 1:
        ci = stats.t.interval(0.95, len(values)-1, loc=mean, scale=stats.sem(values))
    else:
        ci = (mean, mean)
    return mean, std, ci[0], ci[1], len(values)

print("\n" + "="*80)
print("3-WAY COMPARISON: STATISTICAL ANALYSIS")
print("="*80)

stats_dict = {'config': {'num_seeds': NUM_SEEDS, 'seeds': SEEDS}, 'thresholds': {}}

for threshold in THRESHOLDS:
    print(f"\n📊 {threshold:.1%} Accuracy Threshold:")
    print("-" * 80)
    
    g_epochs = [r['threshold_epochs'][threshold] for r in all_grokked_results]
    m_epochs = [r['threshold_epochs'][threshold] for r in all_memorized_results]
    r_epochs = [r['threshold_epochs'][threshold] for r in all_random_results]
    
    g_mean, g_std, g_ci_low, g_ci_high, g_n = calc_stats(g_epochs)
    m_mean, m_std, m_ci_low, m_ci_high, m_n = calc_stats(m_epochs)
    r_mean, r_std, r_ci_low, r_ci_high, r_n = calc_stats(r_epochs)
    
    if g_mean:
        print(f"  Grokked transfer:   {g_mean:.1f} ± {g_std:.1f} epochs (N={g_n}/{NUM_SEEDS})")
    if m_mean:
        print(f"  Memorized transfer: {m_mean:.1f} ± {m_std:.1f} epochs (N={m_n}/{NUM_SEEDS})")
    if r_mean:
        print(f"  Random baseline:    {r_mean:.1f} ± {r_std:.1f} epochs (N={r_n}/{NUM_SEEDS})")
    
    # Key comparisons
    print(f"\n  Comparisons:")
    if g_mean and m_mean:
        ratio_gm = m_mean / g_mean
        print(f"    Grokked vs Memorized:  {ratio_gm:.2f}x (grokked is {ratio_gm:.2f}x faster)")
    if g_mean and r_mean:
        ratio_gr = r_mean / g_mean
        print(f"    Grokked vs Random:     {ratio_gr:.2f}x (grokked is {ratio_gr:.2f}x faster)")
    if m_mean and r_mean:
        ratio_mr = r_mean / m_mean
        print(f"    Memorized vs Random:   {ratio_mr:.2f}x (memorized is {ratio_mr:.2f}x faster)")
    
    stats_dict['thresholds'][threshold] = {
        'grokked': {'mean': g_mean, 'std': g_std, 'ci': [g_ci_low, g_ci_high], 'values': g_epochs},
        'memorized': {'mean': m_mean, 'std': m_std, 'ci': [m_ci_low, m_ci_high], 'values': m_epochs},
        'random': {'mean': r_mean, 'std': r_std, 'ci': [r_ci_low, r_ci_high], 'values': r_epochs}
    }

# Save
with open(f'{EXPERIMENT_DIR}/results/3way_stats.json', 'w') as f:
    json.dump(stats_dict, f, indent=2)

print(f"\n✓ Saved 3-way statistics")

## Visualization: 3-Way Comparison

In [ ]:
# Prepare data
grokked_curves = np.array([r['test_accuracies'] for r in all_grokked_results])
memorized_curves = np.array([r['test_accuracies'] for r in all_memorized_results])

# Pad random curves
random_curves_list = [r['test_accuracies'] for r in all_random_results]
max_len = max(len(c) for c in random_curves_list)
random_curves = np.array([list(c) + [c[-1]]*(max_len-len(c)) for c in random_curves_list])

grokked_mean = grokked_curves.mean(axis=0)
grokked_std = grokked_curves.std(axis=0)

memorized_mean = memorized_curves.mean(axis=0)
memorized_std = memorized_curves.std(axis=0)

random_mean = random_curves.mean(axis=0)
random_std = random_curves.std(axis=0)

print("✓ Prepared curves for visualization")

In [ ]:
# Create comprehensive 3-way figure
fig = plt.figure(figsize=(22, 14))
gs = fig.add_gridspec(3, 3, hspace=0.3, wspace=0.3)

# Row 1: Individual condition plots
ax1 = fig.add_subplot(gs[0, 0])
epochs_g = np.arange(len(grokked_mean))
ax1.plot(epochs_g, grokked_mean, 'b-', linewidth=2.5, label='Grokked Transfer')
ax1.fill_between(epochs_g, grokked_mean-grokked_std, grokked_mean+grokked_std, alpha=0.3, color='blue')
for t in THRESHOLDS:
    ax1.axhline(t, color='red', linestyle='--', alpha=0.3)
ax1.set_xlabel('Epoch')
ax1.set_ylabel('Test Accuracy')
ax1.set_title('Grokked Transfer', fontweight='bold', fontsize=14)
ax1.grid(True, alpha=0.3)
ax1.set_ylim([0, 1.05])

ax2 = fig.add_subplot(gs[0, 1])
epochs_m = np.arange(len(memorized_mean))
ax2.plot(epochs_m, memorized_mean, 'purple', linewidth=2.5, label='Memorized Transfer')
ax2.fill_between(epochs_m, memorized_mean-memorized_std, memorized_mean+memorized_std, alpha=0.3, color='purple')
for t in THRESHOLDS:
    ax2.axhline(t, color='red', linestyle='--', alpha=0.3)
ax2.set_xlabel('Epoch')
ax2.set_ylabel('Test Accuracy')
ax2.set_title('Memorized Transfer (Control)', fontweight='bold', fontsize=14)
ax2.grid(True, alpha=0.3)
ax2.set_ylim([0, 1.05])

ax3 = fig.add_subplot(gs[0, 2])
epochs_r = np.arange(len(random_mean))
ax3.plot(epochs_r, random_mean, 'orange', linewidth=2.5, label='Random Baseline')
ax3.fill_between(epochs_r, random_mean-random_std, random_mean+random_std, alpha=0.3, color='orange')
for t in THRESHOLDS:
    ax3.axhline(t, color='red', linestyle='--', alpha=0.3)
ax3.set_xlabel('Epoch')
ax3.set_ylabel('Test Accuracy')
ax3.set_title('Random Baseline', fontweight='bold', fontsize=14)
ax3.grid(True, alpha=0.3)
ax3.set_ylim([0, 1.05])

# Row 2: Direct comparisons
ax4 = fig.add_subplot(gs[1, :])
# Zoom to transfer epochs for fair comparison
zoom = TRANSFER_EPOCHS
ax4.plot(epochs_g, grokked_mean, 'b-', linewidth=3, label='Grokked Transfer', alpha=0.8)
ax4.plot(epochs_m, memorized_mean, color='purple', linewidth=3, label='Memorized Transfer', alpha=0.8)
ax4.plot(epochs_r[:zoom], random_mean[:zoom], 'orange', linewidth=3, label='Random Baseline', alpha=0.8)

for t in THRESHOLDS:
    ax4.axhline(t, color='red', linestyle='--', alpha=0.3, linewidth=1)
    ax4.text(zoom*0.98, t+0.01, f'{t:.1%}', fontsize=9, color='red', ha='right')

ax4.set_xlabel('Epoch', fontsize=13)
ax4.set_ylabel('Test Accuracy', fontsize=13)
ax4.set_title(f'3-Way Comparison (First {zoom:,} Epochs)', fontweight='bold', fontsize=15)
ax4.legend(fontsize=12, loc='lower right')
ax4.grid(True, alpha=0.3)
ax4.set_ylim([0, 1.05])

# Row 3: Box plots for key threshold (99.9%)
ax5 = fig.add_subplot(gs[2, 0])
threshold_999 = 0.999
box_data = []
labels = []
if threshold_999 in stats_dict['thresholds']:
    g_vals = [v for v in stats_dict['thresholds'][threshold_999]['grokked']['values'] if v is not None]
    m_vals = [v for v in stats_dict['thresholds'][threshold_999]['memorized']['values'] if v is not None]
    r_vals = [v for v in stats_dict['thresholds'][threshold_999]['random']['values'] if v is not None]
    
    if g_vals:
        box_data.append(g_vals)
        labels.append('Grokked')
    if m_vals:
        box_data.append(m_vals)
        labels.append('Memorized')
    if r_vals:
        box_data.append(r_vals)
        labels.append('Random')

if len(box_data) > 0:
    bp = ax5.boxplot(box_data, labels=labels, patch_artist=True, showmeans=True)
    colors = ['lightblue', 'plum', 'lightsalmon']
    for patch, color in zip(bp['boxes'], colors[:len(box_data)]):
        patch.set_facecolor(color)

ax5.set_ylabel('Epochs to 99.9%', fontsize=12)
ax5.set_title('Epochs to 99.9% Accuracy', fontweight='bold', fontsize=14)
ax5.grid(True, alpha=0.3, axis='y')

# Speedup comparison
ax6 = fig.add_subplot(gs[2, 1])
if threshold_999 in stats_dict['thresholds']:
    g_m = stats_dict['thresholds'][threshold_999]['grokked']['mean']
    m_m = stats_dict['thresholds'][threshold_999]['memorized']['mean']
    r_m = stats_dict['thresholds'][threshold_999]['random']['mean']
    
    if g_m and m_m and r_m:
        # Normalize to grokked (grokked = 1.0x)
        speedups = [1.0, g_m/m_m, g_m/r_m]
        labels_speed = ['Grokked\n(baseline)', 'Memorized', 'Random']
        colors_speed = ['blue', 'purple', 'orange']
        
        bars = ax6.bar(labels_speed, speedups, color=colors_speed, alpha=0.7)
        ax6.set_ylabel('Relative Speed (1.0 = Grokked)', fontsize=12)
        ax6.set_title('Relative Speed to 99.9%\n(Higher = Slower)', fontweight='bold', fontsize=14)
        ax6.grid(True, alpha=0.3, axis='y')
        ax6.axhline(1.0, color='red', linestyle='--', linewidth=1, alpha=0.5)
        
        # Add value labels
        for bar, val in zip(bars, speedups):
            ax6.text(bar.get_x() + bar.get_width()/2, bar.get_height() + 0.2,
                    f'{val:.1f}x', ha='center', fontsize=11, fontweight='bold')

# Summary table
ax7 = fig.add_subplot(gs[2, 2])
ax7.axis('off')
summary_lines = [
    "3-WAY COMPARISON SUMMARY",
    "="*45,
    f"Seeds: N={NUM_SEEDS}",
    "",
    "Epochs to 99.9% (mean ± std):"
]

if threshold_999 in stats_dict['thresholds']:
    for cond, name in [('grokked', 'Grokked'), ('memorized', 'Memorized'), ('random', 'Random')]:
        m = stats_dict['thresholds'][threshold_999][cond]['mean']
        s = stats_dict['thresholds'][threshold_999][cond]['std']
        n = stats_dict['thresholds'][threshold_999][cond].get('values', [])
        n_reached = len([v for v in n if v is not None])
        if m:
            summary_lines.append(f"  {name:11s}: {m:6.0f} ± {s:5.0f} ({n_reached}/{NUM_SEEDS})")
        else:
            summary_lines.append(f"  {name:11s}: NOT REACHED")

summary_lines.extend([
    "",
    "Key Finding:",
    "  Grokked transfers generalize",
    "  much faster than memorized",
    "  models, showing that it's the",
    "  GENERALIZING mechanisms that",
    "  enable effective transfer."
])

summary_text = "\n".join(summary_lines)
ax7.text(0.5, 0.5, summary_text, fontsize=10, family='monospace',
         verticalalignment='center', horizontalalignment='center',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.3))

plt.savefig(f'{EXPERIMENT_DIR}/figures/3way_comparison.png', dpi=200, bbox_inches='tight')
print(f"✓ Saved 3-way comparison figure")
plt.show()

## Save Complete Results

In [ ]:
complete_results = {
    'grokked_results': all_grokked_results,
    'memorized_results': all_memorized_results,
    'random_results': all_random_results,
    'statistics': stats_dict,
    'memorized_model_info': {
        'epoch': memorized_epoch,
        'train_loss': memorized_train_loss,
        'test_loss': memorized_test_loss
    },
    'total_time_hours': total_time / 3600
}

torch.save(complete_results, f'{EXPERIMENT_DIR}/results/complete_3way_results.pth')
print(f"✓ Saved complete 3-way results")

print("\n" + "="*80)
print("✅ 3-WAY EXPERIMENT COMPLETE!")
print("="*80)
print(f"\nResults: {EXPERIMENT_DIR}")
print(f"Runtime: {total_time/3600:.2f} hours")
print(f"\nKey finding: Compare grokked vs memorized to confirm")
print(f"that GENERALIZED mechanisms (not just any training) enable transfer!")